In [73]:
!pip install mediapipe
!pip install numpy


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import json
from numpy import dot
from numpy.linalg import norm

In [37]:
model_path = 'mobilenet_v3_small_075_224_embedder.tflite'

In [58]:
BaseOptions = mp.tasks.BaseOptions
ImageEmbedder = mp.tasks.vision.ImageEmbedder
ImageEmbedderOptions = mp.tasks.vision.ImageEmbedderOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = ImageEmbedderOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    quantize=True,
    running_mode=VisionRunningMode.IMAGE)

embeddings = {}

with ImageEmbedder.create_from_options(options) as embedder:
    for root, dirs, files in os.walk('cartoon\\train'):       
        for file in files:
            image_path = os.path.join(root, file)
            try:
                # Embed the current image
                current_mp_image = mp.Image.create_from_file(image_path)
                current_embedding_result = embedder.embed(current_mp_image)

                embeddings[image_path] = current_embedding_result.embeddings[0].embedding.tolist()

            except RuntimeError as e:
                continue

In [59]:
with open("embeddings.txt", "w") as fp:
    json.dump(embeddings, fp)

In [63]:
data = {}

with open('embeddings.txt', 'r') as fp:
    data = json.load(fp)

In [67]:
list(data.keys())[:5]

['cartoon\\train\\donald\\donald_1.png',
 'cartoon\\train\\donald\\donald_10.png',
 'cartoon\\train\\donald\\donald_100.gif',
 'cartoon\\train\\donald\\donald_101.png',
 'cartoon\\train\\donald\\donald_102.jpeg']

In [70]:
def cos_sim(a, b):
    return dot(a, b)/(norm(a)*norm(b))

In [72]:
cosines = {}

with ImageEmbedder.create_from_options(options) as embedder:
    query_mp_image = mp.Image.create_from_file("cartoon\\test\\mickey\\0-4256_baby-clipart-disney-baby-mickey-mouse-transparent-background.jpg")
    query_embedding_result = embedder.embed(query_mp_image).embeddings[0].embedding.tolist()

    for i in data.keys():
        cosines[i] = cos_sim(query_embedding_result, data[i])

for i in sorted(cosines.items(), key=lambda x:x[1], reverse=True)[:10]:
    print(i)

('cartoon\\train\\mickey\\mickey_329.png', 0.8334171079686072)
('cartoon\\train\\mickey\\mickey_5.png', 0.8222460876267234)
('cartoon\\train\\mickey\\mickey_529.png', 0.8191190224977545)
('cartoon\\train\\mickey\\mickey_377.jpg', 0.8151130428540543)
('cartoon\\train\\mickey\\mickey_481.jpg', 0.81410606919268)
('cartoon\\train\\mickey\\mickey_276.png', 0.812734189088538)
('cartoon\\train\\mickey\\mickey_468.png', 0.8117458002774546)
('cartoon\\train\\mickey\\mickey_277.png', 0.811173131686667)
('cartoon\\train\\mickey\\mickey_131.png', 0.8097303702412663)
('cartoon\\train\\mickey\\mickey_353.png', 0.8092662380565608)
